In [1]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
import math
import numpy as np
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

In [3]:
!pip install keras-xlnet

     |████████████████████████████████| 1.0MB 1.4MB/s 
  Created wheel for keras-xlnet: filename=keras_xlnet-0.18.0-cp36-none-any.whl size=44299 sha256=5b88b4e313d7ee47217b3262a28d1eca85a8bef1e0720030cc1ea0c1f86b53d1
  Stored in directory: /root/.cache/pip/wheels/26/fa/2d/68fed1da83d148f8cceea78242835efdd9bdd201620cba67b1
  Created wheel for keras-transformer: filename=keras_transformer-0.31.0-cp36-none-any.whl size=13385 sha256=609f10bf94269e3d3dc41877a18b82a4c6584adbe37bbc71401cf673dde7766d
  Stored in directory: /root/.cache/pip/wheels/a3/c5/9a/5a5130240be614a7a6fa786765d7692ae97f82601e2161bb56
  Created wheel for keras-transformer-xl: filename=keras_transformer_xl-0.11.0-cp36-none-any.whl size=20658 sha256=33e931d9c87f4ed4aa7721baddd6ca1939d229a6b704b98317a34a95bb47b8b9
  Stored in directory: /root/.cache/pip/wheels/ad/8d/1d/d93406806db7a917d1d59054477c1473db1e16fbd789f6dbc6
  Created wheel for keras-trans-mask: filename=keras_trans_mask-0.3.0-cp36-none-any.whl size=4206 sha256=758

In [4]:
!pip install keras-bert

  Created wheel for keras-bert: filename=keras_bert-0.80.0-cp36-none-any.whl size=37923 sha256=7e7b38d9fc3b2e02e931418571da043c9d925abd92f3416af2e690eff51a05da
  Stored in directory: /root/.cache/pip/wheels/63/dc/87/3260cb91f3aa32c0f85c5375429a30c8fd988bbb48f5ee21b0
Successfully built keras-bert


In [5]:
import os
import numpy as np
import pandas as pd
from keras_xlnet.backend import keras
from keras_bert.layers import Extract
from keras_xlnet import PretrainedList, get_pretrained_paths
from keras_xlnet import Tokenizer, load_trained_model_from_checkpoint, ATTENTION_TYPE_BI
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


In [0]:
EPOCH = 10
BATCH_SIZE = 128
SEQ_LEN = 120
MODEL_NAME = 'imdb.h5'

In [0]:
#importing the dataset
train_df = pd.read_csv("/content/drive/My Drive/Movie_Reviews/RNN/train_cleaned.csv")
#train_df, valid_df =  train_test_split(train_df, random_state=0, test_size=0.2)
#print(train_df.shape, valid_df.shape)
test_df = pd.read_csv("/content/drive/My Drive/Movie_Reviews/RNN/test_cleaned.csv")

In [8]:
train_df = train_df.sample(frac=1).reset_index(drop=True)
train_df.head()

,reviews,pos_or_neg
0,love trap must see independent film sat watch ...,1
1,guess go prove joe baker anything buck concept...,0
2,cinderella beautiful young woman treated cruel...,1
3,movie ok like mindless action corny acting sma...,1
4,movie simply far long far repetitive male nudi...,0


In [0]:
#valid_df = valid_df.sample(frac=1).reset_index(drop=True)
#valid_df.head()

In [10]:
#test_df = test_df.sample(frac=1).reset_index(drop=True)
test_df.head()

,reviews,pos_or_neg
0,went saw movie last night coaxed friend mine i...,1
1,actor turned director bill paxton follows prom...,1
2,recreational golfer knowledge sport history pl...,1
3,saw film sneak preview delightful cinematograp...,1
4,bill paxton taken true story u golf open made ...,1


In [11]:
# Load pretrained model
paths = get_pretrained_paths(PretrainedList.en_cased_base)
tokenizer = Tokenizer(paths.vocab)
model = load_trained_model_from_checkpoint(
    config_path=paths.config,
    checkpoint_path=paths.model,
    batch_size=BATCH_SIZE,
    memory_len=0,
    target_len=SEQ_LEN,
    in_train_phase=False,
    attention_type=ATTENTION_TYPE_BI,
)


# Build classification model
last = Extract(index=-1, name='Extract')(model.output)
dense = keras.layers.Dense(units=256, activation='relu', name='Dense')(last)
dropout = keras.layers.Dropout(rate=0.5, name='Dropout')(dense)
output = keras.layers.Dense(units=1, activation='sigmoid', name='Sigmoid')(dropout)
model = keras.models.Model(inputs=model.inputs, outputs=output)
model.summary()

433643520/433638019 [==============================] - 4s 0us/step












Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        (None, 120)          0                                            
__________________________________________________________________________________________________
Embed-Token (EmbeddingRet)      [(None, 120, 768), ( 24576000    Input-Token[0][0]                
__________________________________________________________________________________________________
Masking (CreateMask)            (None, 120)          0           Input-Token[0][0]                
________________________________________________________________________________________________

In [0]:
model.load_weights('/content/drive/My Drive/Movie_Reviews/RNN/weigths.best.xlnet.hdf5')

In [0]:
# Read data
class DataSequence(keras.utils.Sequence):

    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __len__(self):
        return (len(self.y) + BATCH_SIZE - 1) // BATCH_SIZE

    def __getitem__(self, index):
        s = slice(index * BATCH_SIZE, (index + 1) * BATCH_SIZE)
        return [item[s] for item in self.x], self.y[s]

In [0]:
def generate_sequence(dataframe):
  tokens = []
  classes = []
  for _, row in dataframe.iterrows():
    text = row['reviews']
    cls = row['pos_or_neg']
    if not isinstance(text, str) or cls not in [0.0, 1.0]:
      continue
    encoded = tokenizer.encode(text)[:SEQ_LEN - 1] #Tokenize the sentence for XLNet and removes more than maximum length
    encoded = [tokenizer.SYM_PAD] * (SEQ_LEN - 1 - len(encoded)) + encoded + [tokenizer.SYM_CLS] #Pads the sentence upto maximim length and adds the classifier token at the last
    tokens.append(encoded)
    classes.append(int(cls))
  tokens, classes = np.array(tokens), np.array(classes)
  segments = np.zeros_like(tokens)
  segments[:, -1]=1 #Telling what is sentence and classifier symbol in segment encoding
  lengths = np.zeros_like(tokens[:, :1])
  return DataSequence([tokens, segments, lengths], classes)


In [0]:
train_seq = generate_sequence(train_df)
#valid_seq = generate_sequence(valid_df)
test_seq = generate_sequence(test_df)

In [16]:
model.compile(optimizer=keras.optimizers.Adam(lr=1e-7), loss='binary_crossentropy', metrics=['accuracy'])



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:
#Declaring filepath for saving weigths
filepath = "/content/drive/My Drive/Movie_Reviews/RNN/weigths.best.xlnet.hdf5"
""
#creating a checkpoint
checkpoint = ModelCheckpoint(filepath,monitor='val_acc',verbose=1,save_best_only = True, mode = 'max')

In [0]:
model.fit_generator(generator=train_seq, validation_data=test_seq, epochs=2, callbacks=[checkpoint])


Epoch 1/2
 25/196 [==>...........................] - ETA: 49:47 - loss: 0.3003 - acc: 0.8778

In [0]:
# Evaluation
# Use dev set because the results of test set is unknown
results = model.predict_generator(test_seq, verbose=True)#.argmax(axis=-1)

In [0]:
temp = model.evaluate_generator(test_seq)

In [0]:
temp

[0.4591212721157074, 0.800439999961853]

In [0]:
max(results), min(results)

(array([0.9996497], dtype=float32), array([7.4893236e-05], dtype=float32))

In [0]:
from sklearn import metrics
metrics.roc_auc_score(valid_df['pos_or_neg'], results)

0.9567626093122797

In [0]:
prediction = results.argmax(axis=-1)

In [0]:
np.argmax(results, axis=-1)

array([0, 0, 0, ..., 0, 0, 0])

In [0]:
prediction = results.copy()
prediction[prediction>0.5] = 1 
prediction[prediction<=0.5] = 0 

In [0]:
metrics.accuracy_score(test_df['pos_or_neg'], prediction)

NameError: ignored